In [1]:
import SimpleITK as sitk
import numpy as np

# File paths
study_folder_id = "LUNG1-001"
image_file = "CT.nrrd"
mask_file = "mask.nrrd"

# Load the image and mask
image = sitk.ReadImage(image_file)
mask = sitk.ReadImage(mask_file)

# Convert to numpy arrays
image_array = sitk.GetArrayFromImage(image)
mask_array = sitk.GetArrayFromImage(mask)

# Get voxel spacing and origin
spacing = image.GetSpacing()  # Physical spacing in (x, y, z)
origin = image.GetOrigin()  # Physical origin in (x, y, z)

# Compute the bounding box for the mask
bb_filter = sitk.LabelStatisticsImageFilter()
bb_filter.Execute(image, mask)
current_bb = bb_filter.GetBoundingBox(1)  # Get bounding box for label 1

# Extract bounding box coordinates
x_min, x_max = current_bb[0], current_bb[1]
y_min, y_max = current_bb[2], current_bb[3]
z_min, z_max = current_bb[4], current_bb[5]

# Compute the center of the bounding box in voxel space
x_center_voxel = (x_min + x_max) / 2
y_center_voxel = (y_min + y_max) / 2
z_center_voxel = (z_min + z_max) / 2

# Convert the center coordinates to physical space
x_center_phys = origin[0] + x_center_voxel * spacing[0]
y_center_phys = origin[1] + y_center_voxel * spacing[1]
z_center_phys = origin[2] + z_center_voxel * spacing[2]

# Compute the diameter in physical space
x_size_phys = (x_max - x_min) * spacing[0]
y_size_phys = (y_max - y_min) * spacing[1]
z_size_phys = (z_max - z_min) * spacing[2]
diameter_phys = (x_size_phys + y_size_phys + z_size_phys) / 3  # Approximation as average

# Print the results
print(f"study_folder_id: {study_folder_id}")
print(f"coordX: {x_center_phys:.6f}")
print(f"coordY: {y_center_phys:.6f}")
print(f"coordZ: {z_center_phys:.6f}")
print(f"diameter_mm: {diameter_phys:.6f}")


study_folder_id: LUNG1-001
coordX: 80.566406
coordY: -196.351562
coordZ: -456.500000
diameter_mm: 80.221354
